In [1]:
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
from bs4 import BeautifulSoup

## Importing csv files to dataframes

In [172]:
beers_file = "Resources/beers.csv"
beers_df = pd.read_csv(beers_file)
beers_df.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [171]:
breweries_file = "Resources/breweries.csv"
breweries_df = pd.read_csv(breweries_file)
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


## Windows Path

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Mac Path

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Scraping Data from Brewers Association Website

In [147]:
url = 'https://www.brewersassociation.org/statistics-and-data/state-craft-beer-stats/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [216]:
#                           NOT WORKING 

#State cards with data to scrape
states_cards = soup.find('section', class_='site-content wide')
states = states_cards.find_all('div', class_='stat-container')


#Set up lists to append into
state_list = []
million_dollars_list = []
breweries_per_capita_list = []
barrels_produced_list = []
gallons_per_adult_list = []





In [217]:
for state in states:
    states = states_cards.find('div', class_='stat-container')
    state_name = states.find('h1').text.strip()
    state_list.append(state_name)
    #million dollars is the first one
    million_dollars = states_cards.find('span', class_='total').find('span', class_='count').text.strip()
    million_dollars_list.append(million_dollars)
    breweries_per_capita = states_cards.find('span', class_="bpc").find('span', class_= 'count').text.strip()
    breweries_per_capita_list.append(breweries_per_capita)
    #barrels produced is the second one
    barrels_produced = states_cards.find('span', class_='total').find('span', class_='count').text.strip()
    barrels_produced_list.append(barrels_produced)
    gallons_per_adult = states_cards.find('span', class_='per-capita').find('span', class_='count').text.strip()
    gallons_per_adult_list.append(gallons_per_adult)

In [218]:
state_list

['Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama']

In [182]:
million_dollars_list

['758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758',
 '758']

In [183]:
breweries_per_capita_list

['1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1',
 '1.1']

In [166]:
barrels_produced_list

['758', '758', '758', '758', '758', '758']

In [167]:
gallons_per_adult_list

['1.1', '1.1', '1.1', '1.1', '1.1', '1.1']

In [ ]:
url = 'https://www.brewersassociation.org/statistics-and-data/state-craft-beer-stats/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#State cards with data to scrape
states = soup.find_all('div', class_='stat-container')
#Locations for data to scrape
state_name = states.find_all('h1')
million_dollars = states.find_all('span', class_='total').find('span', class_='count')[0]
breweries_per_capita = states.find_all('span', class_="bpc").find('span', class_= 'count')
barrels_produced = states.find_all('span', class_='total').find('span', class_='count')[1]
gallons_per_adult = states.find_all('span', class_='per-capita').find('span', class_='count')
#Set up lists to append into
state_list = []
million_dollars_list = []
breweries_per_capita_list = []
barrels_produced_list = []
gallons_per_adult_list = []


for state in states:
    name = state_name.text.strip()
    state_list.append(state)
    money = million_dollars.text.strip()
    million_dollars_list.apppend(money)
    breweries = breweries_per_capita.text.strip()
    breweries_per_capita_list.apppend(breweries)
    barrels = barrels_produced.text.strip()
    barrels_produced_list.apppend(barrels)
    gallons = gallons_per_adult.text.strip()
    gallons_per_adult_list.apppend(gallons)

## Transforming Data  (example code for what we will do later)

In [ ]:
 # Create a filtered dataframe from specific columns
#premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
#premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
#premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
                                                          "Premises Name": "premise_name",
                                                          "County ID Code": "county_id"})

# Clean the data by dropping duplicates and setting the index
#premise_transformed.drop_duplicates("id", inplace=True)
#premise_transformed.set_index("id", inplace=True)

#premise_transformed.head()

In [ ]:
#county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
#county_transformed = county_df[county_cols].copy()

# Rename the column headers
#county_transformed = county_transformed.rename(columns={"ID": "id",
                                                         "County Name (Licensee)": "county_name",
                                                         "License Count": "license_count",
                                                         "County ID Code": "county_id"})

# Set index
#county_transformed.set_index("id", inplace=True)

#county_transformed.head()

## Connecting to postgres

In [ ]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True) 

In [ ]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True) 